Import Library and data

In [1]:
import openpyxl
import copy
from openpyxl.utils import get_column_letter
from geopy.distance import geodesic
import random

In [2]:
path = "../data/Buildings.xlsx"
def load_data():
    workbook=openpyxl.load_workbook(path)
    worksheet=workbook["Sheet1"]
    buildings=[]
    for cell in list(worksheet.columns)[4][1:]:
        start_POS = cell.value.find("POLYGON (") + 10
        end_POS = cell.value.find("))")#获取第三列的数据
        buildings_str=cell.value[start_POS:end_POS]
        polygon = buildings_str.split(',')
        buildings.append(polygon)
    workbook.close()
    return buildings

polygon_list=load_data()

In [3]:
class MBR:
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

    def __repr__(self):
        result = "("+str( self.x_min)+", " + str(self.y_min) + " " + str(self.x_max) + ", " + str(self.y_max)+")"
        return result

**Task1 Q1** Write a program to compute the spatial extent of dataset D. The spatial extent of D is the MBR of all polygons in D. Report the MBR of D.

In [4]:
#Task 1 Question 1
def cal_mbr_for_all(polygon_list):
    MBR_List=[]
    point_min_x=float("inf")
    point_min_y=float("inf")
    point_max_x=float("-inf")
    point_max_y=float("-inf")
    for polygon in polygon_list:
        temp = cal_mbr(polygon)
        MBR_List.append(temp)
        point_min_x = min(point_min_x, temp.x_min)
        point_min_y = min(point_min_y, temp.y_min)
        point_max_x = max(point_max_x, temp.x_max)
        point_max_y = max(point_max_y, temp.y_max)
    MBR_for_all=MBR(point_min_x, point_min_y, point_max_x,point_max_y)
    return MBR_for_all, MBR_List
def cal_mbr(polygon):
    local_min_x=float("inf")
    local_min_y=float("inf")
    local_max_x=float("-inf")
    local_max_y=float("-inf")
    for polygon_point in polygon:
            p_x, p_y = polygon_point.strip().split(" ")
            p_x = float(p_x)
            p_y = float(p_y)
            local_min_x = min(local_min_x, p_x)
            local_min_y = min(local_min_y, p_y)
            local_max_x = max(local_max_x, p_x)
            local_max_y = max(local_max_y, p_y)
    result=MBR(local_min_x, local_min_y, local_max_x,local_max_y)
    return result

In [5]:

MBR_for_all, MBR_result=cal_mbr_for_all(polygon_list)
print(MBR_result[1])
print("MBR of all Polygons is " +str(MBR_for_all))


(113.9336246, 22.3193894 113.9349546, 22.3200398)
MBR of all Polygons is (113.9310037, 22.1973011 114.3763554, 22.5069962)


**Task1 Q2** Create dataset D’ which adds an MBR column for the polygon in each record in D. Output the spreadsheet file for the new dataset D’.

In [6]:
#Task 1 Question 2
mbr_path = "../data/Buildings_MBR.xlsx"
# def copy_xslx(source,dest):
#     wb = openpyxl.load_workbook(source)
#     wb2 = openpyxl.Workbook()
#     sheetnames = wb.sheetnames
#     for sheetname in sheetnames:
#         print(sheetname)
#         sheet = wb[sheetname]
#         sheet2 = wb2.create_sheet(sheetname)
#         sheet2.sheet_properties.tabColor = sheet.sheet_properties.tabColor
#         wm = list(sheet.merged_cells)
#         if len(wm) > 0:
#             for i in range(0, len(wm)):
#                 cell2 = str(wm[i]).replace('(<CellRange ', '').replace('>,)', '')
#                 sheet2.merge_cells(cell2)
#         for i, row in enumerate(sheet.iter_rows()):
#             sheet2.row_dimensions[i+1].height = sheet.row_dimensions[i+1].height
#             for j, cell in enumerate(row):
#                 sheet2.column_dimensions[get_column_letter(j+1)].width = sheet.column_dimensions[get_column_letter(j+1)].width
#                 sheet2.cell(row=i + 1, column=j + 1, value=cell.value)
#                 source_cell = sheet.cell(i+1, j+1)
#                 target_cell = sheet2.cell(i+1, j+1)
#                 target_cell.fill = copy.copy(source_cell.fill)
#                 if source_cell.has_style:
#                     target_cell._style = copy.copy(source_cell._style)
#                     target_cell.font = copy.copy(source_cell.font)
#                     target_cell.border = copy.copy(source_cell.border)
#                     target_cell.fill = copy.copy(source_cell.fill)
#                     target_cell.number_format = copy.copy(source_cell.number_format)
#                     target_cell.protection = copy.copy(source_cell.protection)
#                     target_cell.alignment = copy.copy(source_cell.alignment)
#     if 'Sheet' in wb2.sheetnames:
#         del wb2['Sheet']
#     wb2.save(dest)
#     wb.close()
#     wb2.close()
#     print('Copy done.')

def add_col(data,dest,tag,col):
    data_sheet = openpyxl.load_workbook(dest)
    booksheet = data_sheet.worksheets[0]
    for i, row in enumerate(booksheet.iter_rows()):
        if i==0:
            booksheet.cell(row=i + 1, column=col, value=tag)
        else:
            booksheet.cell(row=i + 1, column=col, value=str(data[i-1]))
    data_sheet.save(dest)
    data_sheet.close()
    print('Insert done.')
# copy_xslx(path,mbr_path)
add_col(MBR_result,path,"MBR",6)

Insert done.


**Task1 Q3**  Let n be the resolution for recursive decomposition of the space as defined by the spatial extent of D. What are the sizes (in cm by cm) of the smallest Peano cells for n= 16, 23 and 28 respectively? Show your calculation steps. Please also discuss which resolution value is suitable for D.

In [7]:
# Task 1 Question 3
def cal_dist(min_x, min_y, max_x, max_y):
    x_dist= geodesic((min_y,min_x), (min_y,max_x)).m*100 #convert to cm
    y_dist = geodesic((min_y,min_x), (max_y,min_x)).m*100 #convert to cm
    return x_dist,y_dist
def cal_smallest_peano(n, x_dist,y_dist):
    x=x_dist / 2**n
    y=y_dist / 2**n
    return x,y

In [8]:
x_dist,y_dist=cal_dist(MBR_for_all.x_min,MBR_for_all.y_min,MBR_for_all.x_max,MBR_for_all.y_max)
print("x: ",x_dist," y: ",y_dist)
x_12,y_12 = cal_smallest_peano(12, x_dist,y_dist)
print("When n = 12, the x for smallest peano is: " + str(x_12),", y is",str(y_12))
x_16,y_16 = cal_smallest_peano(16, x_dist,y_dist)
print("When n = 16, the x for smallest peano is: " + str(x_16),", y is",str(y_16))
x_20,y_20 = cal_smallest_peano(20, x_dist,y_dist)
print("When n = 20, the x for smallest peano is: " + str(x_20),", y is",str(y_20))


x:  4592407.166791219  y:  3429410.4542472675
When n = 12, the x for smallest peano is: 1121.1931559548875 , y is 837.2584116814618
When n = 16, the x for smallest peano is: 70.07457224718047 , y is 52.32865073009136
When n = 20, the x for smallest peano is: 4.379660765448779 , y is 3.27054067063071


**Task 2 Q1** Write a program to generate the base-5 z-value for each polygon, for n = 16, 23 and 28 respectively. We use only one z-value for each object based on its MBR. Add three columns of z-values to D’ for these three different resolution levels. Output the spreadsheet file with the new columns.

In [9]:
# Task 2 Question 1
def cal_z_value(resolution_level, mbr_t):
    level = 0
    local_x_min =MBR_for_all.x_min
    local_y_min = MBR_for_all.y_min
    local_x_max = MBR_for_all.x_max
    local_y_max = MBR_for_all.y_max
    z_value=""

    while level < resolution_level :
        y_mid = local_y_min+((local_y_max - local_y_min) / 2)
        x_mid = local_x_min+((local_x_max - local_x_min) / 2)

        if mbr_t.y_max <= y_mid and mbr_t.y_min >=local_y_min and mbr_t.x_max <= x_mid and mbr_t.x_min>=local_x_min:
            z_value += "1"
            local_y_max = y_mid
            local_x_max = x_mid
        elif mbr_t.y_max <= local_y_max and mbr_t.y_min >= y_mid and mbr_t.x_max <= x_mid and mbr_t.x_min>=local_x_min:
            z_value += "2"
            local_y_min = y_mid
            local_x_max = x_mid
        elif mbr_t.y_max <= y_mid and mbr_t.y_min >=local_y_min and mbr_t.x_max <= local_x_max and mbr_t.x_min >= x_mid:
            z_value += "3"
            local_x_min = x_mid
            local_y_max = y_mid
        elif mbr_t.y_max <= local_y_max and mbr_t.y_min >= y_mid and mbr_t.x_max <= local_x_max and mbr_t.x_min >= x_mid:
            z_value += "4"
            local_y_min = y_mid
            local_x_min = x_mid
        else:
            break
        level +=  1
    return z_value

In [10]:
z_value_12 = []
z_value_16 = []
z_value_20 = []
for mbr in MBR_result:
    z_value_12.append(cal_z_value(12,mbr))
    z_value_16.append(cal_z_value(16,mbr))
    z_value_20.append(cal_z_value(20,mbr))

add_col(z_value_12,path,"Z-value 12",7)
add_col(z_value_16,path,"Z-value 16",8)
add_col(z_value_20,path,"Z-value 20",9)


Insert done.
Insert done.
Insert done.


**Task 2 Q2** For each object, compare the sizes of the Peano cells for the same object using the above 3 resolution numbers and analyze your findings. Remember that the Peano cells for an object should be as tight as possible. Your discussions should reveal insights about choosing the proper resolution level for an application. If you see any issues with using just one z-value for one object, discuss possible solutions.


In [11]:
# Task 2 Question 2
count_12_16=0
count_12_20=0
count_16_20=0
for i,mbr in enumerate(MBR_result):
    x_dist,y_dist=cal_dist(mbr.x_min,mbr.y_min,mbr.x_max,mbr.y_max)
    num_12 = len(z_value_12[i])
    x_12,y_12 = cal_smallest_peano(num_12, x_dist,y_dist)
    num_16  = len(z_value_16[i])
    x_16,y_16 = cal_smallest_peano(num_16, x_dist,y_dist)
    num_20 = len(z_value_20[i])
    x_20,y_20 = cal_smallest_peano(num_20, x_dist,y_dist)
    if x_12!=x_16 or y_12!=y_16 :
        print("Peano for Polygon ",i," is different for 12 and 16")
        count_12_16+=1
    if x_12!=x_20 or y_12!=y_20:
        print("Peano for Polygon ",i," is different for 12 and 20")
        count_12_20+=1
    if x_16!=x_20 or y_16!= y_20:
        print("Peano for Polygon ",i," is different for 16 and 20")
        count_16_20+=1
print("There is in total ",count_12_16," for 12 and 16")
print("There is in total ",count_12_20," for 12 and 20")
print("There is in total ",count_16_20," for 16 and 20")

Peano for Polygon  19255  is different for 12 and 16
Peano for Polygon  19255  is different for 12 and 20
Peano for Polygon  22144  is different for 12 and 16
Peano for Polygon  22144  is different for 12 and 20
Peano for Polygon  22259  is different for 12 and 16
Peano for Polygon  22259  is different for 12 and 20
Peano for Polygon  22882  is different for 12 and 16
Peano for Polygon  22882  is different for 12 and 20
Peano for Polygon  26036  is different for 12 and 16
Peano for Polygon  26036  is different for 12 and 20
Peano for Polygon  26037  is different for 12 and 16
Peano for Polygon  26037  is different for 12 and 20
Peano for Polygon  30369  is different for 12 and 16
Peano for Polygon  30369  is different for 12 and 20
Peano for Polygon  30448  is different for 12 and 16
Peano for Polygon  30448  is different for 12 and 20
Peano for Polygon  31133  is different for 12 and 16
Peano for Polygon  31133  is different for 12 and 20
Peano for Polygon  31239  is different for 12 

**Task3 Q1**  Write a program to perform window queries using two approaches: (i) by exhaustively checking every object in the dataset; and (ii) by using z-values you generate in Task 2 for the above three n values

In [17]:
# Task 3 Question 1
def is_contain(mbr1, mbr2):
    return True  if mbr1.x_min >= mbr2.x_min and mbr1.x_max <= mbr2.x_max and mbr1.y_min >= mbr2.y_min and mbr1.y_max <= mbr2.y_max else False
def is_contain_by_z(z1,z2):
    if len(z1) < len(z2):
        return False
    if z1[0: len(z2)] == z2:
        return True
    return False
def exhaustive_check(target_mbr):
    inside_obj=0
    for mbr_t in MBR_result:
        if is_contain(mbr_t,target_mbr):
            inside_obj+=1
    return inside_obj
def z_value_check(z,r):
    res_list=[]
    target=[]
    if r==12:
        target=z_value_12
    elif r==16:
        target=z_value_16
    elif r==20:
        target=z_value_20
    for j,z_t in enumerate(target):
        if is_contain_by_z(z_t,z):
            res_list.append(MBR_result[j])
    return res_list
def window_query_check(query):
    num_obj=exhaustive_check(query)
    z_value_q_12 = cal_z_value(12,query)
    z_value_q_16 = cal_z_value(16,query)
    z_value_q_20 = cal_z_value(20,query)
    # for z value first check
    list_obj_12 = z_value_check(z_value_q_12,12)
    list_obj_16 = z_value_check(z_value_q_16,16)
    list_obj_20 = z_value_check(z_value_q_20,20)
    #check x,y coordinate
    num_obj_12=0
    num_obj_16=0
    num_obj_20=0
    for mbr_t in list_obj_12:
        if is_contain(mbr_t,query):
            num_obj_12+=1
    for mbr_t in list_obj_16:
        if is_contain(mbr_t,query):
            num_obj_16+=1
    for mbr_t in list_obj_20:
        if is_contain(mbr_t,query):
            num_obj_20+=1

    print("There are ",num_obj," objects in this window query")
    print("For exhaustive search, the number of object inside is ",num_obj)
    print("For z-value search (resolution=12), the number of object inside is ",num_obj_12)
    print("For z-value search (resolution=16), the number of object inside is ",num_obj_16)
    print("For z-value search (resolution=20), the number of object inside is ",num_obj_20)


In [18]:
 #Task 3 question2
for i in range(20):
    x_min = random.uniform(MBR_for_all.x_min,MBR_for_all.x_max)
    x_max = random.uniform(x_min,MBR_for_all.x_max)
    y_min = random.uniform(MBR_for_all.y_min,MBR_for_all.y_max)
    y_max = random.uniform(y_min,MBR_for_all.y_max)
    query=MBR(x_min, y_min, x_max, y_max)
    print("----------Window Query ",i+1,"----------")
    window_query_check(query)

----------Window Query  1 ----------
There are  0  objects in this window query
For exhaustive search, the number of object inside is  0
For z-value search (resolution=12), the number of object inside is  0
For z-value search (resolution=16), the number of object inside is  0
For z-value search (resolution=20), the number of object inside is  0
----------Window Query  2 ----------
There are  0  objects in this window query
For exhaustive search, the number of object inside is  0
For z-value search (resolution=12), the number of object inside is  0
For z-value search (resolution=16), the number of object inside is  0
For z-value search (resolution=20), the number of object inside is  0
----------Window Query  3 ----------
There are  0  objects in this window query
For exhaustive search, the number of object inside is  0
For z-value search (resolution=12), the number of object inside is  0
For z-value search (resolution=16), the number of object inside is  0
For z-value search (resolutio